In [1]:
import sys, os, time, requests, re
from bs4 import BeautifulSoup
import numpy as np
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from process_travis import *

In [3]:
from utility import *

In [4]:
# create travis obj
process_Travis = ProcessTravisCIBuilds("WMlSAymr_DTc5XHe5mCj4w")

In [5]:
# seglearn latest build list
build_list = process_Travis.get_build_details('notthatanonymous/healthcareai-py', 50)

In [6]:
def process_logs(build_list):
        '''
        :param build_list:
        :return:
        '''
        run = 0
        data_list = []
        for build_id, build_desc in build_list.items():

            run += 1
            status = build_desc['status']
            while status not in ['passed', 'errored', 'failed']:
                time.sleep(30)
                status = process_Travis.get_build_info(build_id)

            print(f"{build_id} completed with status {status}.")

            if status == 'passed':
                for job_id in build_list[build_id]['job_list']:
                    job_details = process_Travis.get_job_details(job_id)
                    try:
                        score = re.findall(r"^Score:\s(\d+\.\d+)\s", job_details['log'], re.MULTILINE)[0]
                    except:
                            score = np.nan
                    data_list.append(list(np.append(job_details['job_config'].split('_'),
                                                    [int(run), round(float(score), 6),
                                                     job_details['processing_time']])))
            else:
                run -= 1
        #print(data_list)
        return data_list

In [7]:
import pandas as pd

df = pd.DataFrame(process_logs(build_list), columns=['OS', 'Python', 'Hardware', 'Run', 'Score', 'Processing_Time'])

266756828 completed with status passed.
266756823 completed with status passed.
266756818 completed with status passed.
266756816 completed with status passed.
266756815 completed with status passed.
266756812 completed with status passed.
266756810 completed with status passed.
266756803 completed with status passed.
266756801 completed with status passed.
266756800 completed with status passed.
266756798 completed with status passed.
266756794 completed with status passed.
266756792 completed with status passed.
266756785 completed with status passed.
266756782 completed with status passed.
266756779 completed with status passed.
266756773 completed with status passed.
266756771 completed with status passed.
266756768 completed with status passed.
266756761 completed with status passed.
266756758 completed with status passed.
266756754 completed with status passed.
266756752 completed with status passed.
266756748 completed with status passed.
266756744 completed with status passed.


In [8]:
df["Project"] = ['healthcareai-py'] * df.shape[0]

In [9]:
df.shape

(400, 7)

In [10]:
df.head()

,OS,Python,Hardware,Run,Score,Processing_Time,Project
0,Linux-Xenial,3.8,amd64,1.0,0.87,58.0,healthcareai-py
1,Linux-Xenial,3.6,amd64,1.0,0.885,64.0,healthcareai-py
2,Linux-Xenial,3.7,amd64,1.0,0.86,55.0,healthcareai-py
3,Linux-Bionic,3.7,amd64,1.0,0.845,47.0,healthcareai-py
4,Linux-Focal,3.7,amd64,1.0,0.88,49.0,healthcareai-py


In [11]:
#df['Score'] = df['Score'].astype(float)/100

In [12]:
df.head()

,OS,Python,Hardware,Run,Score,Processing_Time,Project
0,Linux-Xenial,3.8,amd64,1.0,0.87,58.0,healthcareai-py
1,Linux-Xenial,3.6,amd64,1.0,0.885,64.0,healthcareai-py
2,Linux-Xenial,3.7,amd64,1.0,0.86,55.0,healthcareai-py
3,Linux-Bionic,3.7,amd64,1.0,0.845,47.0,healthcareai-py
4,Linux-Focal,3.7,amd64,1.0,0.88,49.0,healthcareai-py


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   OS               400 non-null    object
 1   Python           400 non-null    object
 2   Hardware         400 non-null    object
 3   Run              400 non-null    object
 4   Score            400 non-null    object
 5   Processing_Time  400 non-null    object
 6   Project          400 non-null    object
dtypes: object(7)
memory usage: 22.0+ KB


In [14]:
write_df(df = df, file_name='healthcareai-py.csv')

In [15]:
# # remove IBM CPU rows from rrcf
# rrcf_df = pd.read_csv('../output/rrcf.csv')
# rrcf_df.head(10)

In [16]:
# rrcf_df = rrcf_df[rrcf_df.Hardware != 's390x']

In [44]:
# rrcf_df.head(10)

In [45]:
# write_df(df = rrcf_df, file_name='rrcf.csv')